In [ ]:
import re
import pandas as pd
import requests

import time #modulo para controlar los tiempos y los sleep (evitar el error 429)

#DEFINO LOS HEADERS: 
headers = {
"X-RapidAPI-Key": "47608e078dmsha9428889aafcc0ap1afc77jsn6e04b24d521c", #MariCarmen
#X-RapidAPI-Key": "2583d39c0emshe321f51d17d9328p10cfb0jsn38ed4dc93da2"  #Zahara
"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

#DEFINO LOS GENEROS 
genres = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

sleep_time = 5

#SACAR TODAS LAS URLS: 

for genre in genres:
  
    for year in range(1990, 2023):
       
        for page in range(1, 10):
            url = f"https://moviesdatabase.p.rapidapi.com/titles?year={year}&genre={genre}&page={page}"
            sleep_time
            response = requests.get(url, headers=headers)
            #SI LA RESPUESTA ES 200 ENTONCES COMIENZO A HACER TODO EL LOOP DE LINKS Y EXTRAER DATOS: 
            if response.status_code == 200:
                # Procesamos la respuesta da data aqui, lo convertimos a JSONy extraemos la información:
                datos = response.json()
                lista_de_diccionarios = datos["results"]
                lista_peliculas = []

                for datos in lista_de_diccionarios: 
                    
                    #lista que usamos para ir appendeando toda la información de abajo /cada peli y será una lista "de paso"
                    lista_consumible = []
                    # aqui creamos las variables con las "rutas " que hemos ido buscando arriba /dentro del json 
                    tipo = datos["titleType"]["text"]
                    texto = datos["titleText"]["text"]
                    #para que no me de error el NONE de aquellas pelis que no tengan mes o año. añado el check de: 
                    release_date = datos.get("releaseDate")
                    if release_date:
                        mes = release_date.get("month")
                        anio = release_date.get("year")
                    else:
                        mes = None
                        anio = None
                    #hasta aqui el control del mes y año 
                    id = datos["id"]

                    if tipo == "Movie" or tipo =="Short":
                        lista_consumible.append(tipo)
                        lista_consumible.append(texto)
                        lista_consumible.append(mes)
                        lista_consumible.append(anio)
                        lista_consumible.append(id)
                        sleep_time

                        #Si la var está, entonces lo appendeamos a la tupla. Tiene q estar aqui dentro porque si esta fuera, irá creando tuplas vacías. 
                        lista_peliculas.append(tuple(lista_consumible))
                        sleep_time
                    else: 
                        pass
                # CONFIRMAME QUE ESTA PASANDO ...
                print(f"Received data for {genre} in {year}, page {page}")

            #SI LA RESPUESTA NO ES 200. TELL ME SOMETHING I DONT KNOW
            else:
                print(f"Failed to retrieve data for {genre} in {year}, page {page}. Status Code: {response.status_code}")
                #añadido este IF para printear cual es el error 429 (too many requests)
                if "error" in response.text:
                    error_message = response.json().get("error")
                    print(f"Error Message: {error_message}")

            print("...")
        
            #HE METIDO EL SLEEP PARA CONTROLAR UN POCO PERO NO SE SI VA 
            time.sleep(sleep_time)

In [ ]:
lista_peliculas

In [ ]:
df = pd.DataFrame(lista_peliculas)

In [ ]:
df